# Ilara (*Queue*) DMA (Datu Mota Abstraktua)

<img src="../img/Queue.jpg" alt="Queue" style="width: 800px;"/>

# Ilara (*Queue*) DMA (Datu Mota Abstraktua)

* FIFO (*First-In First-Out*) motako elementu sorta lineala eta dinamikoa.
    * Gehitzen den lehenengo elementua, ateratzen den lehenengoa.
    * Ezin da lehenengo elementuaren *atzetik* dagoen elementurik atzitu.
* Inplementazio/erabilpenaren arabera, tamaina maximoa izan dezakete
* Erabilpenak: ekoizle/produktore - kontsumitzaile arteko *buffering*-a

## Oinarrrizko eragiketak:
* Hutsik dagoeneko ilara berria **sortu/hasieratu**
* **enqueue**: elementu berri bat gehitu
* **front**: lehenengo elementua kontsultatu
* **dequeue**: lehenengo elementua atera
* **len**: elementu kopurua
* **isEmpty**: hutsik dagoen kontsultatu
   * [bool()](https://docs.python.org/3/library/stdtypes.html#truth) ere erabil dezakegu &rarr; **len**
* **isFull**: beteta dagoen kontsultatu (tamaina maximoa duten ilaretan)


<img src="../img/Queue2.png" alt="Queue" style="width: 900px;"/>

## Queue klasea inplementatzen I - zerrenda mugagabeak

* `Queue.enqueue` &rarr; `list.append`

* `Queue.front` &rarr; `list[0]`

* `Queue.dequeue` &rarr; `list.pop(0)`

* `len(Queue)` &rarr; `len(list)`

* `iter(Queue)` $\not \Rightarrow$ `iter(list)`

In [ ]:
class Queue(object):

    def __init__(self,it=()):
        self.z = list(it)
    
    def enqueue(self,value):
        self.z.append(value)

    def front(self):
        return self.z[0]
    
    def dequeue(self):
        return self.z.pop(0)
    
    def __len__(self):
        return len(self.z)
    
    def isFull(self):
        return False

    def __iter__(self):
        return (self.dequeue() for _ in range(len(self)))
    
    def __iter__(self):
        for _ in range(len(self)) :
            yield self.dequeue()
    
    def __iter__(self):
        while self :
            yield self.dequeue()

    # ez da zuzena...
    #def __iter__(self):
    #    return iter(self.z)



In [ ]:
q = Queue(range(10))
print(list(q))
q.enqueue("kaixo")
print(list(q))

q = Queue(range(10))
print(*q)
q.enqueue("kaixo")
print(*q)

In [ ]:
q = Queue()
for i in "aeiou":
    q.enqueue(i)
    print(f'len: {len(q)}')
print('-----')
while q :
    x = q.dequeue()
    print(f'dequeue: {x} len: {len(q)}')

In [ ]:
q = Queue()
for i in "aeiou":
    q.enqueue(i)
    print(f'len: {len(q)}')
print('-----')
for x in q:
    print(f'dequeue: {x} len: {len(q)}')

In [ ]:
q = Queue(range(10))
for x in q :
    print(x)
    if x == 5 or x==6 or x==7:
        q.enqueue(f'-->{x}')
        print('txertatu, luzera berria:',len(q))


### `str` funtzio berezia...

* `print(Queue("aeiou"))` :

```
['a', 'e', 'i', 'o', 'u']
```

edo

```
['u', 'o', 'i', 'e', 'a']
```

In [ ]:
class Queue(object):

    def __init__(self,it=()):
        self.z = list(it)
    
    def enqueue(self,value):
        self.z.append(value)

    def front(self):
        return self.z[0]
    
    def dequeue(self):
       return self.z.pop(0)
    
    def __len__(self):
        return len(self.z)
    
    def isFull(self):
        return False
    
    def __iter__(self):
        while self :
            yield self.dequeue()
    
    def __str__(self):
        return '-->'+str(self.z[::-1])+'-->'

In [ ]:
print(Queue("aeiou"))

In [ ]:
print(Queue([1,2,3,'kaixo',1.3]))

### `repr` funtzio berezia...

* `q == eval(repr(q))` bete dadin saiatu
   * `Queue([., ., ., ...])` erabili
   * `==` &rarr; `__eq__()`

In [ ]:
class Queue(object):

    def __init__(self,it=()):
        self.z = list(it)
    
    def enqueue(self,value):
        self.z.append(value)

    def front(self):
        return self.z[0]
    
    def dequeue(self):
       return self.z.pop(0)
    
    def __len__(self):
        return len(self.z)
    
    def isFull(self):
        return False
    
    def __iter__(self):
        while self :
            yield self.dequeue()
    
    def __str__(self):
        return str(self.z)
    
    def __repr__(self):
        return f'Queue({repr(self.z)})'

    def __eq__(self,other):
        return type(other) == Queue and self.z == other.z

In [ ]:
q = Queue([1,2,3,'kaixo',1.3,(5,6),[64,24,(23,234,265)]])
print(q)
print(repr(q))

In [ ]:
q == eval(repr(q))

### Inplementazioari buruz...

* Python-eko zerrendak array-etan oinarrituak daude

* Elementuak memoria bloke jarrai batean gordetzen dira:
<img src="../img/ArrayList1.png" alt="Queue" style="width: 300px;"/>   

* Array-an elementu berri batentzat lekurik ez badago, array handiago bat sortzen da, elementu zahar guztiak array berrian kopiatuz
<img src="../img/ArrayList2.png" alt="Queue" style="width: 450px;"/>   

* Elementu bat ezabatzean, bere atzetik dauden guztiak ezkerrera desplazatu:
<img src="../img/ArrayList3.png" alt="Queue" style="width: 550px;"/>   

### Beraz...

* `Queue.enqueue(value)` $\equiv$ `list.append(value)`) &rarr; batzuetan array berriak sortu eta elementu guztiak kopiatu beharko ditu.
   * <u>Gutxitan</u> gertatuko da.
   * **Batazbestean**, ez da kostu handia izango

* `Queue.dequeue()` $\equiv$ `list.pop(0)` &rarr; array-ko elementu guztiak desplazatu behar ditu
   * <u>Beti</u> gertatuko da
   * **kostu handiegia**

### &rarr; Ez da inplementazio egokia

## Queue klasea inplementatzen II - zerrenda lotuak

* Elementu bakoitzak *daki* hurrengo elementua non dagoen:
* DMA implementatzeko, bi objektu mota definitu behar dira:
* **Queue (goiburua)**: egitura printzipala. Bere eremuak:
   * **len**: elementu kopurua
   * **front**: lehenengo elementuaren erreferentzia edo `None`(hutsik badago)
   * **back**: azkeneko elementuaren erreferentzia edo `None` (hutsik badago)
* **element**: balioak gordetzeko egitura. Bere eremuak:
   * **value**: ilaran gordetako balioa
   * **next**: hurrengo elementuaren erreferentzia, edo `None` (azkenengoa bada)

* Elementuak dituen ilara:

<img src="../img/LinkedList-Queue1.png" alt="Queue" style="width: 800px;"/>

* Elementurik gabeko ilara:

<img src="../img/LinkedList-Queue1b.png" alt="Queue" style="width: 266px;"/>

### Klaseak definitzen I : Bi klase

In [1]:
class Queue(object):
    def __init__(self, it=()):
        self.len = 0
        self.front = None
        self.back = None
        for x in it :
            self.enqueue(x)

In [2]:
class element(object):
    def __init__(self,value,next_element=None):
        self.value = value
        self.next = next_element

### Klaseak definitzen Ii : Klase bakarra

* Klase baten barnean, beste klase bat defini dezakegu

In [1]:
class Queue(object):
    
    class element(object):
        def __init__(self,value,next=None):
            self.value = value
            self.next = next
        
    def __init__(self, it=()):
        self.len = 0
        self.front = None
        self.back = None
        for x in it :
            self.enqueue(x)

Bi datu mota definitu ditugu, `Queue` eta `Queue.element`:

In [5]:
q = Queue()
print(q)
e = Queue.element(7)
print(e)
print(q.len, q.front, q.back)
print(e.value, e.next)

0 None None
7 None


### `len()`

In [6]:
class Queue(object):
    class element(object):
        def __init__(self,value,next=None):
            self.value = value
            self.next = next
        
    def __init__(self, it=()):
        self.len = 0
        self.front = None
        self.back = None
        for x in it :
            self.enqueue(x)
            
    def __len__(self):
        return self.len

### `enqueue(value)`

<img src="../img/LinkedList-Queue2.png" alt="Queue" style="width: 1030px;"/>

1. `value` balioa duen elementu berri bat sortu
1. Azken elementuaren `next` eguneratu
1. `back` eta `len` eguneratu

### `enqueue(value)` - Ilara hutsik denean

<img src="../img/LinkedList-Queue2b.png" alt="Queue" style="width: 350px;"/>

1. `value` balioa duen elementu berri bat sortu
1. `front`, `back` eta `len` eguneratu

In [7]:
class Queue(object):
    class element(object):
        def __init__(self,value,next=None):
            self.value = value
            self.next = next
        
    def __init__(self, it=()):
        self.len = 0
        self.front = None
        self.back = None
        for x in it :
            self.enqueue(x)           
            
    def __len__(self):
        return self.len
    
    def enqueue(self,value):
        e = Queue.element(value)
        if self.len == 0 :
            self.front = e
        else :
            self.back.next = e
        self.back = e
        self.len += 1

### `dequeue()`

<img src="../img/LinkedList-Queue3.png" alt="Queue" style="width: 1015px;"/>

1. `front` eta `len` eguneratu
1. Ilaratik ateratako elementuaren balioa bueltatu

### `dequeue()` - Ilaran elementu bakarra dagoenean

<img src="../img/LinkedList-Queue3b.png" alt="Queue" style="width: 350px;"/>

1. `front` , `len` eta `back` eguneratu
1. Ilaratik ateratako elementuaren balioa bueltatu

In [ ]:
class Queue(object):
    class element(object):
        def __init__(self,value,next=None):
            self.value = value
            self.next = next
        
    def __init__(self, it=()):
        self.len = 0
        self.front = None
        self.back = None
        for x in it :
            self.enqueue(x)
            
    def __len__(self):
        return self.len
    
    def enqueue(self,value):
        e = Queue.element(value)
        if self.len == 0 :
            self.front = e
        else :
            self.back.next = e
        self.back = e
        self.len += 1

    def dequeue(self):
        value = self.front.value
        self.len -= 1
        self.front = self.front.next
        if self.len == 0 :
            self.back = None
        return value


### `str` , `repr` eta `==` ...

* `print(Queue("aeiou"))` :

```
['a', 'e', 'i', 'o', 'u']
```

* `print(repr(Queue("aeiou")))` :

```
Queue(['a', 'e', 'i', 'o', 'u'])
```

* `str` , `repr` eta `==` inplementatzea errazagoa litzateke, ilarako elementuak zeharkatu ditzakeen `values()` metodo bat bagenu:

```python
class Queue(object):

    ...

    def __str__(self):
        return str(list(self.values()))
    
    def __repr__(self):
        return f'Queue({list(self.values())})'
    
    def __eq__(self,other):
        return type(other) == Queue and list(self.values()) == list(other.values())    
    
    
```

In [ ]:
class Queue(object):
    
    class element(object):
        def __init__(self,value,next=None):
            self.value = value
            self.next = next
        
    def __init__(self, it=()):
        self.len = 0
        self.front = None
        self.back = None
        for x in it :
            self.enqueue(x)
            
    def values(self):
        e = self.front
        while e != None :
            #print(e.value)
            yield e.value
            e = e.next
            
    def __len__(self):
        return self.len
    
    def enqueue(self,value):
        e = Queue.element(value)
        if self.len == 0 :
            self.front = e
        else :
            self.back.next = e
        self.back = e
        self.len += 1

    def dequeue(self):
        value = self.front.value
        self.len -= 1
        self.front = self.front.next
        if self.len == 0 :
            self.back = None
        return value

In [ ]:
q = Queue(range(10))
print(list(q.values()))
print(*q.values())

In [ ]:
q = Queue()
list(q.values())

In [ ]:
q = Queue()
for i in range(5):
    q.enqueue(i)
    print(list(q.values()))
while q :
    print(q.dequeue(),list(q.values()))

In [ ]:
class Queue(object):
    class element(object):
        def __init__(self,value,next=None):
            self.value = value
            self.next = next
        
    def __init__(self, it=()):
        self.len = 0
        self.front = None
        self.back = None
        for x in it :
            self.enqueue(x)
            
    def values(self):
        e = self.front
        while e :
            yield e.value
            e = e.next
            
    def __str__(self):
        return str(list(self.values()))
    
    def __repr__(self):
        return f'Queue({list(self.values())})'
    
    def __eq__(self,other):
        #return type(other) == Queue and list(self.values()) == list(other.values())
        return type(other) == Queue and len(self)==len(other) and all( x==y for x,y in zip(self.values(),other.values()))
    
    def __len__(self):
        return self.len
    
    def enqueue(self,value):
        e = Queue.element(value)
        if self.len == 0 :
            self.front = e
        else :
            self.back.next = e
        self.back = e
        self.len += 1

    def dequeue(self):
        value = self.front.value
        self.len -= 1
        self.front = self.front.next
        if self.len == 0 :
            self.back = None
        return value

In [ ]:
print(Queue())
print(Queue("aeiou"))

In [ ]:
print(repr(Queue()))
print(repr(Queue("aeiou")))

In [ ]:
Queue("aeiou") == Queue("aeiouxxxx")

In [ ]:
q = Queue("aeiou")
print(repr(q))
q == eval(repr(q))

### `iter` 

In [ ]:
class Queue(object):
    class element(object):
        def __init__(self,value,next=None):
            self.value = value
            self.next = next
        
    def __init__(self, it=()):
        self.len = 0
        self.front = None
        self.back = None
        for x in it :
            self.enqueue(x)

    def front(self):
        return self.front.value
            
    def __iter__(self):
        while self :
            #print(self.dequeue())
            yield self.dequeue()
            
    def __len__(self):
        return self.len
    
    def enqueue(self,value):
        e = Queue.element(value)
        if self.len == 0 :
            self.front = e
        else :
            self.back.next = e
        self.back = e
        self.len += 1

    def dequeue(self):
        value = self.front.value
        self.len -= 1
        self.front = self.front.next
        if self.len == 0 :
            self.back = None
        return value

    def values(self):
        e = self.front
        while e :
            yield e.value
            e = e.next
            
    def __str__(self):
        return str(list(self.values()))
    
    def __repr__(self):
        return f'Queue({list(self.values())})'
    
    def __eq__(self,other):
        return type(other) == Queue and len(self)==len(other) and all( x==y for x,y in zip(self.values(),other.values()))
    

In [ ]:
q = Queue(range(10))
print(q)
print(tuple(q))
print(q)

In [ ]:
q = Queue(range(10))
print(q)
for x in q :
    print(x,q)

### Baina... orain gure `Queue`-ak `values()` metodoa du... nola izkutatu?

* Python-en eremu *pribatuak* deklaratu daitezke:
   * `self.izena` &rarr; eremu publikoa
   * `self.__izena` &rarr; eremu pribatua
   * `self.__izena__` &rarr; eremu publiko berezia
   

In [ ]:
class Froga(object):
    def __init__(self):
        self.x = 'publikoa naiz'
        self.__y = 'pribatua naiz'
        
    def erakutsi_pribatua(self):
        print('Sekretua:',self.__y)
        

In [ ]:
f = Froga()
print(f.x)
# ERROREA
#print(f.__y)
f.erakutsi_pribatua()

* Tira... egia esan eremu pribatuak ez dira horren pribatuak
   * `self.__izena` &rarr; eremu pribatua
   * `self.__izena` &rarr; self.`_DATUMOTA__izena` eremu publikoa
      * Klasearen barnean `self.__izena` erabili daiteke

In [ ]:
# ERROREA
#print(f.__y)
# BAINA HAU EGIN DAITEKE...
print(f._Froga__y)


In [ ]:
class Queue(object):
    class element(object):
        def __init__(self,value,next=None):
            self.value = value
            self.next = next
        
    def __init__(self, it=()):
        self.len = 0
        self.front = None
        self.back = None
        for x in it :
            self.enqueue(x)

    def __values(self):
        e = self.front
        while e :
            yield e.value
            e = e.next
            
    def __str__(self):
        return str(list(self.__values()))
    
    def __repr__(self):
        return f'Queue({list(self.__values())})'
    
    def __eq__(self,other):
        return type(other) == Queue and len(self)==len(other) and all( x==y for x,y in zip(self.__values(),other.__values()))
    
    def __len__(self):
        return self.len
    
    def enqueue(self,value):
        e = Queue.element(value)
        if self.len == 0 :
            self.front = e
        else :
            self.back.next = e
        self.back = e
        self.len += 1

    def dequeue(self):
        value = self.front.value
        self.len -= 1
        self.front = self.front.next
        if self.len == 0 :
            self.back = None
        return value
    
    def __iter__(self):
        while self :
            yield self.dequeue()  

    def front(self):
        return self.front.valu

In [ ]:
q = Queue(range(10))
print(q)
# ERROREA
#list(q.__values())
# Hau ez genuke sekula egin behar...
list(q._Queue__values())
